In [19]:
# ejemplo: consulta del usuario
payload = {
    # ───────────── UBICACIÓN / TIEMPO ─────────────
    "DPTO_MPIO":      11001,     # Código DIVIPOLA (11001 = Bogotá D.C.)
    "ANO_CENSO":      2000,      # Año de la visita del CEED usada para la estimación
    "TRIMESTRE":      4,         # 1-I, 2-II, 3-III, 4-IV
    "ANIO_MESINICIO": 2014,      # Año en que inició la obra (aprox.)  ─► opcional

    # ───────────── CARACTERÍSTICAS GENERALES ─────────────
    "DESTINO2":       1,         # 1=Vivienda, 4=Comercio, etc.
    "ESTADO_ACT":     1,         # Estado actual: 1=En construcción
    "ESTRATO":        4,         # Estrato (0–6)
    "LOCALIDAD2":     11,        # Localidad / Comuna (0 si no aplica)
    "AMPLIACION":     2,         # 1=Sí, 2=No (obra nueva)
    "OB_FORMAL":      2,         # 1=Formal, 2=Informal
    "USO":            1,         # Uso principal (1=Residencial)
    "USO_DOS":        0,         # Uso secundario (0=ninguno)

    # ───────────── SUPERFICIES (m²) ─────────────
    "AREATOTCO2":     10_780,     # Área total construida
    "AREATOTZC":      32_025,     # Área de zonas comunes
    "AREAVENUNI":     70.04,     # Área unitaria de venta
    "AREAVENDIB":     11_755,    # Área vendible total (aprox.)
    "AREA_NOVIS":     14_780,    # m² destinados a No-VIS
    # Desglose por rango de valor:
    "AREA_RANVIVI_1": 0.0,
    "AREA_RANVIVI_2": 0.0,
    "AREA_RANVIVI_3": 0.0,
    "AREA_RANVIVI_4": 3_589.43,
    "AREA_RANVIVI_5": 11_190.57,
    "AREA_RANVIVI_6": 0.0,

    # ───────────── CONTEOS DE UNIDADES ─────────────
    "UNI_DEC_NOVIS":     140,    # Nº viviendas No-VIS
    "UNIDEC_RANVIVI_1":  0,
    "UNIDEC_RANVIVI_2":  0,
    "UNIDEC_RANVIVI_3":  0,
    "UNIDEC_RANVIVI_4":  34,
    "UNIDEC_RANVIVI_5": 106,
    "UNIDEC_RANVIVI_6": 0,
    "NUMUNIDEST":       140,     # Unidades destinadas
    "NUMUNIVEN":        0,       # Unidades en venta
    "NUMUNIXVEN":       17,      # Unidades vendidas
    "NRO_EDIFIC":       1,       # Nº edificios en el proyecto
    "NRO_PISOS":        18,      # Nº pisos

    # ───────────── MANO DE OBRA / COSTOS ─────────────
    "MANO_OBRAP":   1,           # % personal propio (0-100)
    "MANO_OBRAT":   155,         # % contratistas (puede exceder 100 por reporte)
    "MANO_OBRAF":   0.0,         # Costo mano de obra (si se aporta; 0=desconocido)

    # ───────────── VALOR DECLARADO ─────────────
    "PRECIOVTAX":   7_288.93,    # Valor m² (miles COP)  ← ¡variable objetivo en training!
    "TIPOVRDEST":   1,           # 1=Costo, 2=Precio, …

    # ───────────── AVANCE GLOBAL ─────────────
    "CAPITULO":    2,            # Capítulo principal censado (referencia interna)
    "GRADOAVANC":  85,           # % avance total

    # ───────────── PORCENTAJES POR CAPÍTULO (0-100) ─────────────
    "C1_EXCAVACION":     100,
    "C1_CIMENTACION":    100,
    "C1_DESAGUES":       100,
    "C2_ESTRUCTURA":      15,
    "C2_INST_HIDELEC":     0,
    "C2_CUBIERTA":         0,
    "C3_MAMPOSTERIA":      0,
    "C3_PANETE":           0,
    "C4_PISO_ENCHAPE":     0,
    "C4_CARP_METALICA":    0,
    "C4_CARP_MADERA":      29.15,
    "C4_CIELO_RASO":       29.42,
    "C5_VID_CERRAJERIA":   0,
    "C5_PINTURA":          0,
    "C6_REM_EXTERIORES":   0,
    "C6_REM_ACABADOS":     0,
    "C6_ASEO":             0,

    # ───────────── SISTEMA CONSTRUCTIVO & FORMALIDAD ─────────────
    "SIS_CONSTR":        4,      # 1=Tradicional, 4=Industrializado, etc.
    "LIC_RADICADO_SN":   1,      # 1=Sí, 2=No
    "MOVIMIENTO_ENC":    1,      # 1=Movimiento de tierras ejecutado

    # ───────────── RANGOS / BANDERAS ─────────────
    "RANVIVI":            6,     # 1-6: rango de valor principal
}

In [ ]:
import numpy as np
import pandas as pd

# cargar
preproc   = joblib.load("modelos/PruebaTest/feature_pipeline.joblib")
y_scaler  = joblib.load("modelos/PruebaTest/y_scaler.joblib")
model     = joblib.load("modelos/final/XGBoost_model.joblib")


X_new = pd.DataFrame([payload])

# transformar + predecir
X_new_trans   = preproc.transform(X_new)
y_pred_scaled = model.predict(X_new_trans)
price_m2      = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()[0]   # miles COP/m²

print(f"🏷️  Estimación: {price_m2:,.1f} mil COP por m²")


🏷️  Estimación: 4,574.4 mil COP por m²


In [3]:
# cargar
preproc   = joblib.load("modelos/Prueba_II/feature_pipeline.joblib")
y_scaler  = joblib.load("modelos/Prueba_II/y_scaler.joblib")
model     = joblib.load("modelos/Prueba_II/lgbm_reg.joblib")


X_new = pd.DataFrame([payload])

# transformar + predecir
X_new_trans   = preproc.transform(X_new)
y_pred_scaled = model.predict(X_new_trans)
price_m2      = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()[0]   # miles COP/m²

print(f"🏷️  Estimación: {price_m2:,.1f} mil COP por m²")

🏷️  Estimación: 5,168.5 mil COP por m²


c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\JemyC\AppData\Local\Program

In [4]:
# cargar
preproc   = joblib.load("modelos/Prueba_II/feature_pipeline.joblib")
y_scaler  = joblib.load("modelos/Prueba_II/y_scaler.joblib")
model     = joblib.load("modelos/Prueba_II_Best/LGBM_model.joblib")


X_new = pd.DataFrame([payload])

# transformar + predecir
X_new_trans   = preproc.transform(X_new)
y_pred_scaled = model.predict(X_new_trans)
price_m2      = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()[0]   # miles COP/m²

print(f"🏷️  Estimación: {price_m2:,.1f} mil COP por m²")

🏷️  Estimación: 4,363.2 mil COP por m²


c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [5]:
# cargar
preproc   = joblib.load("modelos/PruebaTest/feature_pipeline.joblib")
y_scaler  = joblib.load("modelos/PruebaTest/y_scaler.joblib")
model     = joblib.load("modelos/final/LGBM_model.joblib")


X_new = pd.DataFrame([payload])

# transformar + predecir
X_new_trans   = preproc.transform(X_new)
y_pred_scaled = model.predict(X_new_trans)
price_m2      = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()[0]   # miles COP/m²

print(f"🏷️  Estimación: {price_m2:,.1f} mil COP por m²")

🏷️  Estimación: 4,875.0 mil COP por m²


c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [23]:
# cargar
preproc   = joblib.load("modelos/PruebaTest/feature_pipeline.joblib")
y_scaler  = joblib.load("modelos/PruebaTest/y_scaler.joblib")
model     = joblib.load("modelos/final_models/LGBM_Full_model.joblib")


X_new = pd.DataFrame([payload])

# transformar + predecir
X_new_trans   = preproc.transform(X_new)
y_pred_scaled = model.predict(X_new_trans)
price_m2      = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()[0]   # miles COP/m²

print(f"🏷️  Estimación: {price_m2:,.1f} mil COP por m²")

🏷️  Estimación: 4,875.0 mil COP por m²


c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
import os
from agentAI.agent import *
api_key = os.getenv("DEEPSEEK_API_KEY")

In [ ]:
pdf_path = "cuestionario_visual_053.pdf"
# Uso
agent = AgentEstimacionCEED(api_key)
result = agent.process_ceed_form(pdf_path)

if result:
    print("Datos extraídos:")
    print(json.dumps(result, indent=2))
else:
    print("No se pudo procesar el PDF")

In [ ]:
import joblib
import json

In [4]:
preproc   = joblib.load("modelos/PruebaTest/feature_pipeline.joblib")
y_scaler  = joblib.load("modelos/PruebaTest/y_scaler.joblib")
model     = joblib.load("modelos/final_models/LGBM_Full_model.joblib")

In [6]:
result = json.loads(result)

In [ ]:
X_new = pd.DataFrame([result])

# transformar + predecir
X_new_trans   = preproc.transform(X_new)
y_pred_scaled = model.predict(X_new_trans)
price_m2      = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()[0]   # miles COP/m²

print(price_m2)

6949.625015111051


c:\Users\JemyC\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
